# Piano Accompaniment Arrangement with AccoMontage
This notebook is a showcase and tutorial for
 [AccoMontage](https://arxiv.org/abs/2108.11213), a state-of-the-art piano accompanient arrangement system. 
* The input to the AccoMontage system is a lead sheet of a complete song (i.e., a MIDI file with a melody track and a chord track) with phrase labels.
* In this notebook, we assume the phrase labels of the song to be manually provided.
* The output of the system is a piano accompaniment for the whole piece.

AccoMontage now supports a few new features as follows:
1. Generation with MIDI velocity and pedal control messages.
2. Phrase transition of any length from 1-bar, 2-bar, to 16-bar phrases.
3. Support input of general MIDI with an arbituary number of tracks and arbituarily complex chords (e.g., 9th chords). Yet, we still quantize the chord sequence at 1-beat unit. If the melody is not on the first track, then the melody track index is also requested.
4. Support whole pieces arrangement with intro, interlude, and outro.

## 1. Load dependencies and set up the system
### 1.1 Premisies
* In the first place, please download `checkpoints.zip` [via this link](https://drive.google.com/file/d/1zQ5xds8oeeAlnn_PK5e0PWNKyM7unUFO/view?usp=sharing) (300MB in total, including model checkpoints and pre-processed data).
* After download, please extract the zip file at the working directory. You should now have a `./checkpoints/` folder with relevant pt and npz files inside. 

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import datetime
import warnings
warnings.filterwarnings("ignore")
from AccoMontage import set_premises, load_lead_sheet, phrase_selection, re_harmonization, matrix2leadsheet

phrase_data_dir = 'checkpoints/phrase_data.npz'
edge_weights_dir = 'checkpoints/edge_weights.npz'
checkpoint_dir = 'checkpoints/model_master_final.pt'
pop909_meta_dir = 'checkpoints/pop909_quadraple_meters_index.xlsx'

print('Loading reference data (texture donors) from POP909. This may takes several seconds of time ...')
model, acc_pool, reference_check, params = pre_liminary = set_premises(phrase_data_dir, edge_weights_dir, checkpoint_dir, pop909_meta_dir)

Loading reference data (texture donors) from POP909. This may takes several seconds of time ...


100%|██████████| 16/16 [00:06<00:00,  2.53it/s]


### 1.2 Put it easy
* Let's pack up the whole working pipeline of AccoMontage into a concise function under the name `piano_arrangement`.

In [13]:
def piano_arrangement(SONG_NAME, SEGMENTATION, DEMO_ROOT='demo', PREFILTER=None, SPOTLIGHT=None, RANDOMENESS=0, PICK_UP_BEAT=0.0, TEMPO=100, MELODY_TRACK_ID=0):
    """
    Piano accompaniment arrangement using AccoMontage.
    :param:`SONG_NAME` (str, requested): the name (i.e., MIDI directory) of the input lead sheet.
    :param:`SEGMENTATION` (str, requested): the phrase annotation of the lead sheet. For example, A8A8B8B8 for a 32-bar lead sheet.
    :param:`DEMO_ROOT` (str, optional): the demo root. Default './demo'.
    :param:`PREFILTER` (tuple, optional): pre-filter of accompaniment w.r.t. rhythmic density and voice number. Valid range is [0..4] corresponding to [low .. high]. Default None.
    :param:`SPOTLIGHT` (list, optional): spotlight certain reference pieces as the donor of texture. Valid elements can be POP909 song index (int), song name (str, in Chinese), or artist name (str, in Chinese). Default None.
    :param:`RANDOMENESS` (float, optional): the degree of randomness to be introduced to the phrase selection process. Valid range is [0, 1]. Default 0 (i.e., no randomness).
    :param:`PICK_UP_BEAT` (float, optional): the number of beats in the pick-up measure, requested if there is one. Half beat (0.5) and quater beats (0.25, 0.75) are also supported. Default 0.
    :param:`TEMPO` (int, optional): the BPM to render the accompaniment. Default 100.
    :param:`MELODY_TRACK_ID` (int, optional): The index of the melody track in the input MIDI file. Default 0 (i.e., the first track).
    """

    """step 0: load lead sheet"""
    lead_sheet, chord_roll, phrase_label = load_lead_sheet(DEMO_ROOT, SONG_NAME, SEGMENTATION, PICK_UP_BEAT, MELODY_TRACK_ID)
    #midi_recon = matrix2leadsheet(lead_sheet).write(os.path.join(DEMO_ROOT, SONG_NAME, 'lead_sheet_recon.mid'))

    """step 1: phrase selection (in search for donors of piano textures)"""
    selection = phrase_selection(lead_sheet, phrase_label, reference_check, acc_pool, *params, PREFILTER, SPOTLIGHT, RANDOMENESS)

    """step 2: re-harmonization (chord-texture style transfer)"""
    midi = re_harmonization(lead_sheet, chord_roll, phrase_label, *selection, model, acc_pool, TEMPO)

    """finally, save arrangement result"""
    time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    save_path = os.path.join(DEMO_ROOT, SONG_NAME, f'piano_arrangement_{time}.mid')
    midi.write(save_path)
    print('Result saved at', save_path)

## 2. Arrangement showcase
In the following, we showcase the AccoMontage system with 5 demo pieces. 

### 2.1 Auld Lang Syne
* This is a well-known folk Scottish music with an `A8B8A8B8` phrase structure.
* We set `PREFILTER` for rhythmic density at degree 4 and voice number at degree 2.
* This piece has a pick-up measure of 1 beat hence we set `PICK_UP_BEAT`=1.0.

In [14]:
kwargs = dict({
            #compulsory entries
            'SONG_NAME': 'Auld Lang Syne',  # song name / directory
            'SEGMENTATION': 'A8B8A8B8',     # phrase annotation
            #optional pre-filter
            'PREFILTER': (4, 2),            # set rhythmic density degree 4 and voice number degree 2
            'RANDOMENESS': 0.1,             # introduce a little randomness
            #song meta
            'PICK_UP_BEAT': 1.0,            # there is a one-beat pick-up measure in this song
            'TEMPO': 75,                    # render the arrangement at 75 BPM
            })

piano_arrangement(**kwargs)     # call piano arrangement process

Phrase selection begins: 4 phrases in total. 
	 Set note density filter: (4, 2).


100%|██████████| 3/3 [00:13<00:00,  4.43s/it]


Reference pieces: ['0: 284_天在下雨我在想你', '1: 272_多想留在你身边', '2: 269_外面的世界', '3: 272_多想留在你身边']
Result saved at ./demo\Auld Lang Syne\piano_arrangement_20230529_161802.mid


### 2.2 Castles in the Air
* This is the demo piece in the [AccoMontage paper](https://arxiv.org/abs/2108.11213) with an `A8A8B8B8` phrase structure.
* We set `SPOTLIGHT` on `song_322` and `song_346` of POP909 dataset during phrase selection.
* We set `PREFILTER` for rhythmic density at degree 4 and voice number at degree 1.
* This piece has a pick-up measure of 1 hence we set `PICK_UP_BEAT`=1.0.

In [5]:
kwargs = dict({
            #compulsory entries
            'SONG_NAME': 'Castles in the Air',  # song name / directory
            'SEGMENTATION': 'A8A8B8B8',         # phrase annotation
            #optional pre-filter
            'SPOTLIGHT': [322, 346],            # spotlight on song_322 and song_346 (of POP909) during phrase selection 
            'PREFILTER': (4, 1),                # set rhythmic density degree 4 and voice number degree 1
            #song meta
            'PICK_UP_BEAT': 1.0,                # there is a one-beat pick-up measure in this song
            'TEMPO': 120,                       # render the arrangement at 120 BPM
            })

piano_arrangement(**kwargs)         # call piano arrangement process

Phrase selection begins: 4 phrases in total. 
	 Set note density filter: (4, 1).
	 Refer to [322, 346] as much as possible


100%|██████████| 3/3 [00:20<00:00,  6.80s/it]


Reference pieces: ['0: 322_寂寞沙洲冷', '1: 322_寂寞沙洲冷', '2: 346_平凡之路', '3: 346_平凡之路']
Result saved at ./demo\Castles in the Air\piano_arrangement_20230529_145318.mid


### 2.3 Cuillin Reel
* This is the full demo piece in the [deep music analogy (EC2-VAE) paper](https://arxiv.org/abs/1906.03626) with an `A8B8B8`.
* We set spotlight on the artist `邓丽君` in POP909 dataset during phrase selection.
* This piece has a pick-up measure of 1 beat hence we set `PICK_UP_BEAT`=1.0.

In [9]:
kwargs = dict({
            #compulsory entries
            'SONG_NAME': 'Cuillin Reel',  # song name / directory
            'SEGMENTATION': 'A8B8B8',     # phrase annotation
            #optional pre-filter
            'SPOTLIGHT': ['邓丽君'],       # spotlight on the artist 邓丽君 (Teresa Teng) during phrase selection 
            'RANDOMENESS': 0.2,           # introduce a little randomness
            #song meta
            'PICK_UP_BEAT': 1.0,          # there is a one-beat pick-up measure in this song
            })

piano_arrangement(**kwargs)   # call piano arrangement process

Phrase selection begins: 3 phrases in total. 
	 Set note density filter: None.
	 Refer to ['邓丽君'] as much as possible


100%|██████████| 2/2 [00:12<00:00,  6.43s/it]


Reference pieces: ['0: 707_粉红色的回忆', '1: 707_粉红色的回忆', '2: 707_粉红色的回忆']
Result saved at ./demo\Cuillin Reel\piano_arrangement_20230529_145953.mid


### 2.4 ECNU Alma Mater Music
* This is the Alma Mater Music of East China Normal University. AccoMontage is capable of handling long-term pieces with multiple phrases (16 in this case).
* We set `SPOTLIGHT` on the specific piece `小龙人` in POP909 dataset during phrase selection.
* See [this video](https://zhaojw1998.github.io/accomontage_demo) for our performance demo for the 70th-anniversary celebration of ECNU! 

In [10]:
kwargs = dict({
            #compulsory entries
            'SONG_NAME': 'ECNU Alma Mater Music',                   # song name / directory
            'SEGMENTATION': 'A8A8B8B8C8D8E4F6A8A8B8B8C8D8E4F6',     # phrase annotation
            #optional pre-filter
            'SPOTLIGHT': ['小龙人'],        # spotlight on the piece 小龙人 (Little Dragon Man) during phrase selection 
            'RANDOMENESS': 0.1,             # introduce a little randomness
            #song meta
            'PICK_UP_BEAT': 0,              # there is no pick-up measure
            'TEMPO': 106,                   # render the arrangement at 106 BPM
            })

piano_arrangement(**kwargs)     # call piano arrangement process

Phrase selection begins: 16 phrases in total. 
	 Set note density filter: None.
	 Refer to ['小龙人'] as much as possible


100%|██████████| 15/15 [01:10<00:00,  4.69s/it]


Reference pieces: ['0: 907_小龙人', '1: 907_小龙人', '2: 907_小龙人', '3: 907_小龙人', '4: 907_小龙人', '5: 907_小龙人', '6: 247_和你一样', '7: 265_在深秋', '8: 907_小龙人', '9: 907_小龙人', '10: 907_小龙人', '11: 907_小龙人', '12: 907_小龙人', '13: 907_小龙人', '14: 830_酒醉的探戈2001', '15: 265_在深秋']
Result saved at ./demo\ECNU Alma Mater Music\piano_arrangement_20230529_150642.mid


### 2.5 Let It Be
* This is a well-known pop song by John Lennon and Paul McCartney. AccoMontage is capable of handling whole pieces with intro, interlude, and outro.
* In this case, our input is a general multi-track MIDI and the melody is on the 10-th track. Hence we set `MELODY_TRACK_ID`=9 (counting from 0).

In [11]:
kwargs = dict({
            #compulsory entries
            'SONG_NAME': 'Let It Be',                   # song name / directory
            'SEGMENTATION': 'i4A8B4A8B8x12B4A8B8B4x2',  # phrase annotation
            #optional pre-filter
            'PREFILTER': (3, 4),        # set rhythmic density degree 3 and voice number degree 4
            'RANDOMENESS': 0.1,         # introduce a little randomness
            #song meta
            'MELODY_TRACK_ID': 9,       # melody is on the 10-th track of the input MIDI file
            'TEMPO': 75,                # render the arrangement at 75 BPM
            })

piano_arrangement(**kwargs)     # call piano arrangement process

Phrase selection begins: 11 phrases in total. 
	 Set note density filter: (3, 4).


100%|██████████| 10/10 [00:23<00:00,  2.30s/it]


Reference pieces: ['0: 027_一曲红尘', '1: 413_我', '2: 645_电台情歌', '3: 510_春泥', '4: 425_我们都是好孩子', '5: 908_小乌龟', '6: 684_突然好想你', '7: 014_一个人的精彩', '8: 683_空城', '9: 747_芊芊', '10: 378_忽然好想念']
Result saved at ./demo\Let It Be\piano_arrangement_20230529_151022.mid


## 3. Test with your own music
If you wish to use AccoMontage to arrange piano accompaniments for your own piece, please go for the following steps:
1. Rename your MIDI file as `lead sheet.mid`.
2. Put the renamed file to `./demo/SONG_NAME`, where `SONG_NAME` is the name of your piece.
3. Specify the phrase `SEGMENTATION` for your piece, together with `PICK_UP_BEAT`, `MELODY_TRACK_ID` and `TEMPO` if necessory.
4. Set up `SPOTLIGHT`, `PREFILTER`, and `RANDOMENESS` accoridng to you preference.
5. Put these argumants to the `piano_arrangement` function as shown above, and your arrangement is ready at `./demo/SONG_NAME/piano_arrangement.mid`.